# Prepare data for Training

A number of routines for preparing data inside ML Studio for training

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

#todo: replace file name with the bigger one when script is ready
sales =  pd.DataFrame.from_csv('PE-TRG-Jan-Mar-2017.csv').reset_index()

#remove duplications by summing up quantity by days
sales = pd.pivot_table(sales, values='Quantity', index=['Locationid','PLU','Year','Month','Day'], aggfunc=np.sum).reset_index()

## Raw data properties

In [2]:
def setCountGroup(row):
    if row['counts'] < 10:
        return "1. less than 10"
    elif row['counts'] <= 20:
        return "2. 20 or less"
    elif row['counts'] <= 30:
        return "3. 30 or less"
    else: 
        return "4. more than 30"

positions = sales.groupby(['Locationid','PLU']).size().rename('counts').reset_index()
positions_count = len(positions.index)
locations_count = len(positions.drop_duplicates(['Locationid']).index)
products_count = len(positions.drop_duplicates(['PLU']).index)

counts_groups = positions
counts_groups['segment'] = counts_groups.apply(setCountGroup, axis=1)
counts_groups = counts_groups.groupby('segment').size().rename('counts').reset_index()
counts_groups['%'] = counts_groups.apply(lambda r: r['counts']/positions_count*100, axis=1)

rows_total = len(sales.index)
rows_with_negative = len(sales[sales['Quantity'] < 0].index)
rows_with_zero = len(sales[sales['Quantity'] == 0].index)
rows_with_positive = len(sales[sales['Quantity'] > 0].index)

# ==== output results ====

print("Total unique positions (Location-Recipe) %d" % (positions_count))
print("Total locations %d" % (locations_count))
print("Total recipes %d" % (products_count))

print("Data rows %d" % (rows_total))
print("Negative data %.2f%% (%d)" % (rows_with_negative/rows_total*100, rows_with_negative))
print("Zero data %.2f%% (%d)" % (rows_with_zero/rows_total*100, rows_with_zero))
print("Positive data %.2f%% (%d)" % (rows_with_positive/rows_total*100, rows_with_positive))

counts_groups

Total unique positions (Location-Recipe) 477460
Total locations 517
Total recipes 5652
Data rows 11002942
Negative data 3.47% (382091)
Zero data 1.11% (122375)
Positive data 95.42% (10498476)


,segment,counts,%
0,1. less than 10,216410,45.325263
1,2. 20 or less,78171,16.372262
2,3. 30 or less,44637,9.348846
3,4. more than 30,138242,28.953630


## Prepare data

Set zero values for negative quantity

In [3]:
#fast version of assignment column value (instead of df.apply)
sales.loc[sales['Quantity'] < 0, 'Quantity'] = 0

In [4]:
def creatDate(r):
    return dt.date(int(r['Year']), int(r['Month']), int(r['Day']))

# (!) processing of 11002942 rows takes about 5 mins
sales['Saledate'] = sales.apply(creatDate, axis=1)

In [185]:
_sales = sales.reset_index()#.set_index(['Locationid','PLU','Saledate'])
del _sales['index']

Add previous days chronologically

In [209]:
# todo: remove head()
#positions = _sales.drop_duplicates(['Locationid','PLU'])[['Locationid','PLU']].head(5)

s = _sales.head(5)

def addPreviousDays(p, df, days):
    locationid = p['Locationid']
    plu = p['PLU']
    date_finish = p['Saledate']
    date_start = date_finish - dt.timedelta(days=days)
    # filter sales by position and time frame in days
    position_sales = _sales.loc[(_sales['Locationid']==locationid)
                                &(_sales['PLU']==plu)
                                &(_sales['Saledate']>=date_start)
                                &(_sales['Saledate']<=date_finish)] # at least current row is required to provide the min date
    min_date = position_sales['Saledate'].min()
    
    #add row with starting date if missing
    if min_date > date_start:
        new_row = pd.DataFrame([[locationid,plu,0,0,0,0,date_start]], columns=['Locationid','PLU','Year','Month','Day','Quantity','Saledate'])
        position_sales = position_sales.append(new_row)
    
    position_sales = position_sales.reset_index().set_index('Saledate')
    # Convert the integer timestamps in the index to a DatetimeIndex
    position_sales.index = pd.to_datetime(position_sales.index)
    
    #resample data by days between starting and ending dates
    qty_by_days = position_sales.resample('d').sum().reset_index()
    del qty_by_days['index']
    qty_by_days['Locationid'] = locationid
    qty_by_days['PLU'] = plu
    qty_by_days = qty_by_days.fillna(0)
    
    #transpose resampled data to get dates in columns
    for index in range(1, days+1): # the index is moved +1 to skip the first date in resampled data
        d = date_finish - dt.timedelta(days=index)
        # take the value for day in the past
        # df[].item() - this returns the first element in the Index/Series returned from that selection
        dn = qty_by_days.loc[qty_by_days['Saledate']==d, 'Quantity'].item()
        df.loc[(df['Locationid']==locationid)&(df['PLU']==plu)&(df['Saledate']==date_finish),'Day-'+str(index)] = dn

s.apply(addPreviousDays, args=(s,7,), axis=1)
s

#date_start = sales['Saledate'].min()
#date_finish = sales['Saledate'].max()

C:\Users\agoida\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\agoida\AppData\Local\Continuum\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


,Locationid,PLU,Year,Month,Day,Quantity,Saledate,Day-1,Day-2,Day-3,Day-4,Day-5,Day-6,Day-7,Day-8
0,1,185,2017,1,14,1.0,2017-01-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,185,2017,1,24,2.0,2017-01-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,191,2017,1,1,6.0,2017-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,191,2017,1,2,5.0,2017-01-02,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,191,2017,1,3,2.0,2017-01-03,5.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#take the previous value to add more information (shift - move pointer backward)
qty_by_days['Day-1'] = qty_by_days['Quantity'].shift(1)
qty_by_days['Day-2'] = qty_by_days['Quantity'].shift(2)
qty_by_days['Day-3'] = qty_by_days['Quantity'].shift(3)
qty_by_days['Day-4'] = qty_by_days['Quantity'].shift(4)
qty_by_days['Day-5'] = qty_by_days['Quantity'].shift(5)
qty_by_days['Day-6'] = qty_by_days['Quantity'].shift(6)
qty_by_days['Day-7'] = qty_by_days['Quantity'].shift(7)
#replace NaN with zeros
qty_by_days.fillna(0,inplace=True)

#take weeks averages
def week_avg(df, newColumn, startIndex):
    df[newColumn] = (np.nan_to_num(df['Quantity'].shift(startIndex))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+1))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+2))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+3))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+4))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+5))
                         +np.nan_to_num(df['Quantity'].shift(startIndex+6)))/7
    
week_avg(qty_by_days, 'Week-1', 1)
week_avg(qty_by_days, 'Week-2', 2)
week_avg(qty_by_days, 'Week-3', 3)
week_avg(qty_by_days, 'Week-4', 4)
week_avg(qty_by_days, 'Week-5', 5)

qty_by_days

In [ ]:
qty_by_days.corr()['Quantity']

In [ ]:
qty_by_days[qty_by_days['PLU'] == 1894].to_csv('1894.csv', sep=',')

In [ ]:
import pylab as plt

X = qty_by_days['Quantity']
Y1 = qty_by_days['Day-1']
Y2 = qty_by_days['WeekDay']

#plt.scatter(X,Y1,color='k', s=2)
plt.scatter(X,Y2,color='g', s=2)
plt.show()